In [1]:
import torch
import copy
from trl import SFTTrainer
from transformers import TrainerCallback, Trainer
from peft import get_peft_model_state_dict, set_peft_model_state_dict, LoraConfig, get_peft_model


/Users/vblack/opt/miniconda3/envs/fedllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
import os

# Load the IMDb dataset
dataset = load_dataset('imdb')

# Create a local directory to save the dataset
os.makedirs('dataset', exist_ok=True)

# Save the dataset locally in Arrow format
dataset['train'].save_to_disk('dataset/imdb_train')
dataset['test'].save_to_disk('dataset/imdb_test')
dataset['unsupervised'].save_to_disk('dataset/imdb_unsupervised')

print("Dataset saved locally in the 'dataset' folder in Arrow format.")

Creating CSV from Arrow format: 100%|██████████| 50/50 [00:01<00:00, 43.10ba/s]

Dataset saved locally in the 'dataset' folder.
